# Rossmann Store Sales MSDS Capstone Project

Ben Wilson, Nidhi Shah, Shreyas Adiyodi

## Load Data & Packages

### Load Python Packages

In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import json
import lightgbm as lgb
from datetime import datetime
import matplotlib as mpl
from pandas import datetime
import math
import re
import time
import random
import os
import pickle
import joblib
from pandas.tseries.offsets import *
from operator import *
import warnings 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from tensorflow.keras.models import Sequential
from IPython.display import Image
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from patsy import dmatrices, dmatrix, build_design_matrices, demo_data
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense

/tmp/ipykernel_738/2118618771.py:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime
2023-04-07 19:53:21.741559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 19:53:21.794343: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
#set nicer defaults for plotting.
params = {'legend.fontsize': 'large',
          'figure.figsize': (11.0, 11.0),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
mpl.rcParams.update(params)

#makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

### Load Datasets

In [4]:
#use the pandas `read_csv()` function to read the CSV file into a pandas DataFrame
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_store = pd.read_csv("store.csv")

In [5]:
#view train data to verify no issues
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
#view test data to verify no issues
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [7]:
#view store data to verify no issues
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Perform Data Prep & Cleaning

### Summarization

In [8]:
#define summarization tool
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [9]:
#summarize train data
summarize_dataframe(df_train)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Store,int64,0,1115,1017209.0,,,,558.429727,321.908651,1.0,280.0,558.0,838.0,1115.0
DayOfWeek,int64,0,7,1017209.0,,,,3.998341,1.997391,1.0,2.0,4.0,6.0,7.0
Date,object,0,942,1017209.0,942,2015-07-31,1115,,,,,,,
Sales,int64,0,21734,1017209.0,,,,5773.818972,3849.926175,0.0,3727.0,5744.0,7856.0,41551.0
Customers,int64,0,4086,1017209.0,,,,633.145946,464.411734,0.0,405.0,609.0,837.0,7388.0
Open,int64,0,2,1017209.0,,,,0.830107,0.375539,0.0,1.0,1.0,1.0,1.0
Promo,int64,0,2,1017209.0,,,,0.381515,0.485759,0.0,0.0,0.0,1.0,1.0
StateHoliday,object,0,5,1017209.0,5,0,855087,,,,,,,
SchoolHoliday,int64,0,2,1017209.0,,,,0.178647,0.383056,0.0,0.0,0.0,0.0,1.0


In [10]:
#summarize test data
summarize_dataframe(df_test)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,int64,0,41088,41088.0,,,,20544.5,11861.228267,1.0,10272.75,20544.5,30816.25,41088.0
Store,int64,0,856,41088.0,,,,555.899533,320.274496,1.0,279.75,553.5,832.25,1115.0
DayOfWeek,int64,0,7,41088.0,,,,3.979167,2.015481,1.0,2.0,4.0,6.0,7.0
Date,object,0,48,41088.0,48,2015-09-17,856,,,,,,,
Open,float64,11,2,41077.0,,,,0.854322,0.352787,0.0,1.0,1.0,1.0,1.0
Promo,int64,0,2,41088.0,,,,0.395833,0.489035,0.0,0.0,0.0,1.0,1.0
StateHoliday,object,0,2,41088.0,2,0,40908,,,,,,,
SchoolHoliday,int64,0,2,41088.0,,,,0.443487,0.496802,0.0,0.0,0.0,1.0,1.0


In [11]:
#summarize store data
summarize_dataframe(df_store)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Store,int64,0,1115,1115.0,,,,558.0,322.01708,1.0,279.5,558.0,836.5,1115.0
StoreType,object,0,4,1115.0,4,a,602,,,,,,,
Assortment,object,0,3,1115.0,3,a,593,,,,,,,
CompetitionDistance,float64,3,654,1112.0,,,,5404.901079,7663.17472,20.0,717.5,2325.0,6882.5,75860.0
CompetitionOpenSinceMonth,float64,354,12,761.0,,,,7.224704,3.212348,1.0,4.0,8.0,10.0,12.0
CompetitionOpenSinceYear,float64,354,23,761.0,,,,2008.668857,6.195983,1900.0,2006.0,2010.0,2013.0,2015.0
Promo2,int64,0,2,1115.0,,,,0.512108,0.500078,0.0,0.0,1.0,1.0,1.0
Promo2SinceWeek,float64,544,24,571.0,,,,23.595447,14.141984,1.0,13.0,22.0,37.0,50.0
Promo2SinceYear,float64,544,7,571.0,,,,2011.763573,1.674935,2009.0,2011.0,2012.0,2013.0,2015.0
PromoInterval,object,544,3,571.0,3,"Jan,Apr,Jul,Oct",335,,,,,,,


### Review Data Types

In [12]:
#review data types
df_train.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

### Replace NaN Values

In [13]:
#view unique inputs to df_test open field
unique_inputs = df_test['Open'].unique()
print(unique_inputs)

[ 1. nan  0.]


In [14]:
#replace all nan fields to 0
df_test['Open'] = df_test['Open'].fillna(0)

#confirm all adjusted
unique_inputs = df_test['Open'].unique()
print(unique_inputs)

[1. 0.]


In [15]:
#replace nan values from df_store with 0 to indicate no distance or no promo as do not know
df_store["CompetitionDistance"].fillna(0, inplace=True)
df_store["CompetitionOpenSinceMonth"].fillna(0, inplace=True)
df_store["CompetitionOpenSinceYear"].fillna(0, inplace=True)
df_store["Promo2SinceWeek"].fillna(0, inplace=True)
df_store["Promo2SinceYear"].fillna(0, inplace=True)
df_store["PromoInterval"].fillna(0, inplace=True)

### Convert Fields

In [16]:
#convert date field
df_train['Date'] = pd.to_datetime(df_train['Date'], format='%Y-%m-%d')

In [17]:
#new fields for year, month, day weekday
df_train['Year'] = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month
df_train['Day'] = df_train['Date'].dt.day
df_train['WeekOfYear'] = df_train['Date'].dt.weekofyear

In [18]:
#state holiday conversion
df_train["StateHoliday"] = df_train["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

In [19]:
#merge store and train data
df = pd.merge(df_train, df_store, how='left', on='Store')
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,31,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,31,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,31,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,31,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [20]:
#create season
df['Season'] = df['Date'].map(lambda x: 'Winter' if x.month in [12, 1, 2] 
                               else 'Spring' if x.month in [3, 4, 5] 
                               else 'Summer' if x.month in [6, 7, 8] 
                               else 'Fall')

## EDA

### Superior Stores Comparison

In [ ]:
#chart 1 - Average Sales vs. Average Customers by Store
store_stats = df.groupby('Store')[['Sales', 'Customers']].mean().reset_index()
fig, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(data=store_stats, x='Sales', y='Customers')
plt.title('Average Sales vs. Average Customers by Store', fontsize=10)
plt.xlabel('Average Sales')
plt.ylabel('Average Customers')

#chart 2 - Average Sales vs. Average Customers by StoreType
store_stats = df.groupby('Store')[['Sales', 'Customers']].mean().reset_index()
store_stats = store_stats.merge(df[['Store', 'StoreType']].drop_duplicates(), on='Store')
fig2, ax2 = plt.subplots(figsize=(8, 6))
sns.scatterplot(data=store_stats, x='Sales', y='Customers', hue='StoreType', ax=ax2)
plt.title('Average Sales vs. Average Customers by StoreType', fontsize=10)
plt.xlabel('Average Sales')
plt.ylabel('Average Customers')

#chart 3 - Average Sales vs. Average Customers by StoreType with Store Count
df_counts = df.groupby('StoreType', as_index=False)['Sales', 'Customers', 'Store'].agg({'Sales':'mean', 'Customers':'mean', 'Store':'count'})
df_counts = df_counts.rename(columns={'Store':'Count'})
fig3, ax3 = plt.subplots(figsize=(8, 6))
sns.scatterplot(x='Sales', y='Customers', hue='StoreType', size='Count', sizes=(50,500), data=df_counts)
plt.title('Average Sales vs Average Customers by Store Type', fontsize=10)

#display the charts side by side
fig, axes = plt.subplots(ncols=3, figsize=(20, 6))
axes[0].set_title('Average Sales vs. Average Customers by Store', fontsize=10)
axes[0].set_xlabel('Average Sales')
axes[0].set_ylabel('Average Customers')
sns.scatterplot(data=store_stats, x='Sales', y='Customers', ax=axes[0])

axes[1].set_title('Average Sales vs. Average Customers by StoreType', fontsize=10)
axes[1].set_xlabel('Average Sales')
axes[1].set_ylabel('Average Customers')
sns.scatterplot(data=store_stats, x='Sales', y='Customers', hue='StoreType', ax=axes[1])

axes[2].set_title('Average Sales vs Average Customers by Store Type', fontsize=10)
sns.scatterplot(x='Sales', y='Customers', hue='StoreType', size='Count', sizes=(50,500), data=df_counts, ax=axes[2])
plt.show()

In [ ]:
# group the dataframe by StoreType and calculate the mean of Sales and Customers
grouped = df.groupby('StoreType')[['Sales', 'Customers']].mean()

# create bar charts using matplotlib
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
grouped['Sales'].plot(kind='bar', ax=ax[0])
ax[0].set_title('Average Sales per Store by StoreType')
ax[0].set_xlabel('StoreType')
ax[0].set_ylabel('Average Sales')
grouped['Customers'].plot(kind='bar', ax=ax[1])
ax[1].set_title('Average Customers per Store by StoreType')
ax[1].set_xlabel('StoreType')
ax[1].set_ylabel('Average Customers')
plt.tight_layout()
plt.show()

### Review Sales by Time

In [ ]:
#group by date for avg. sales and % change
average_sales = df_train.groupby('Month')["Sales"].mean()
pct_change_sales = df_train.groupby('Month')["Sales"].sum().pct_change()

fig, (axis1,axis2) = plt.subplots(2,1,sharex=True,figsize=(15,6))

#plot avg. sales over time
ax1 = average_sales.plot(legend=True, ax=axis1, marker='', color="red", title="Avg. Sales Per Month")
ax1.set_xticks(range(len(average_sales)))
ax1.set_xticklabels(average_sales.index.tolist())
ax1.set_ylabel('Sales', size=12)

# plot precent change for sales over time (year-month)
ax2 = pct_change_sales.plot(legend=True, ax=axis2, marker='', title="Sales % Change Per Month")
plt.ylabel("Sales", size=12)
plt.xlabel('Month', size=12)

plt.show()

In [ ]:
# group by date for avg. sales and % change
average_sales = df_train.groupby('WeekOfYear')["Sales"].mean()
pct_change_sales = df_train.groupby('WeekOfYear')["Sales"].sum().pct_change()

# calculate linear fit for average sales over time
x = np.array(range(len(average_sales)))
y = average_sales.values
fit = np.polyfit(x, y, 1)
fit_fn = np.poly1d(fit)

# plot avg. sales over time with trend line
fig, (axis1, axis2) = plt.subplots(2, 1, sharex=True, figsize=(15,6))
ax1 = average_sales.plot(legend=True, ax=axis1, marker='', color="blue", title="Avg. Sales Per Week")
ax1.set_xticks(range(len(average_sales)))
ax1.set_xticklabels(average_sales.index.tolist())
ax1.set_ylabel('Sales', size=12)
ax1.plot(x, fit_fn(x), '-r')  # add trend line in red

# plot percent change for sales over time (year-week)
ax2 = pct_change_sales.plot(legend=True, ax=axis2, marker='', title="Sales % Change Per Week")
plt.ylabel("Sales", size=12)
plt.xlabel('Week Of Year', size=12)

plt.show()

In [ ]:
#review sales and customers by day of week 
fig, (axis1, axis2) = plt.subplots(1, 2, figsize=(10, 4))

sns.barplot(x='DayOfWeek', y='Sales', data=df_train, order=[1, 2, 3, 4, 5, 6, 7], color='blue', ax=axis1)
sns.barplot(x='DayOfWeek', y='Customers', data=df_train, order=[1, 2, 3, 4, 5, 6, 7], color='blue', ax=axis2)

axis1.set_title('Total Sales by Day of Week')
axis2.set_title('Total Customers by Day of Week')
plt.show()

In [ ]:
#evaluate # of closed vs open days
fig, (axis1) = plt.subplots(1,1,figsize=(10,3))
sns.countplot(x='Open', data=df_train, ax=axis1)

### Review Holiday's

In [ ]:
#review state holiday with and without promotion
sns.set(style='whitegrid')

fig, (axis1, axis2) = plt.subplots(1, 2, figsize=(15, 4))

sns.barplot(x='StateHoliday', y='Sales', data=df_train, ax=axis1)
sns.barplot(x='StateHoliday', y='Sales', data=df_train, hue='Promo', ax=axis2)

axis1.set_title('Total Sales by StateHoliday')
axis2.set_title('Total Sales by StateHoliday with Promo')

plt.show()

In [ ]:
#sales by state holiday vs school holiday
sns.set(style='whitegrid')

fig, (axis1, axis2) = plt.subplots(1, 2, figsize=(15, 4))

sns.barplot(x='StateHoliday', y='Sales', data=df_train, ax=axis1)
sns.barplot(x='SchoolHoliday', y='Sales', data=df_train, ax=axis2)

axis1.set_title('Total Sales by StateHoliday')
axis2.set_title('Total Sales by SchoolHoliday')

plt.show()

In [ ]:
# Filter the dataset for the year 2013
df_2013 = df.loc[df['Date'].dt.year == 2013]

# Group the filtered dataset by date and sum the sales
sales_by_date = df_2013.groupby('Date')['Sales'].sum()

# Create a line chart of sales by date
plt.figure(figsize=(20, 6))
plt.plot(sales_by_date.index, sales_by_date.values)

# Add a vertical line for each StateHoliday or SchoolHoliday of 1
holidays = df_2013.loc[(df_2013['StateHoliday'] == 1) | (df_2013['SchoolHoliday'] == 1)]
for date in holidays['Date'].values:
    plt.axvline(date, color='red', linestyle='--')

plt.title('Sales by Date in 2013 with StateHoliday or SchoolHoliday of 1')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

### Promotion Participation & Impact

In [ ]:
#determine if all stores participated in the promotions

#get the unique stores in the dataset
stores = df['Store'].unique()

#check if all stores participated in Promo
if df['Promo'].nunique() == 1:
    all_promo = df['Promo'].unique()[0]
    print(f"All stores participated in Promo: {all_promo}")
else:
    print("Not all stores participated in Promo.")

#check if all stores participated in Promo2
if df['Promo2'].nunique() == 1:
    all_promo2 = df['Promo2'].unique()[0]
    print(f"All stores participated in Promo2: {all_promo2}")
else:
    print("Not all stores participated in Promo2.")

In [ ]:
#compare sales and customers for promo and non-promo stores

#subset the data to include only stores that participated in Promo and those that did not
promo_stores = df[df['Promo'] == 1]
no_promo_stores = df[df['Promo'] == 0]

#calculate the average Sales and average Customers for stores that participated in Promo and those that did not
promo_sales_avg = promo_stores['Sales'].mean()
promo_customers_avg = promo_stores['Customers'].mean()
no_promo_sales_avg = no_promo_stores['Sales'].mean()
no_promo_customers_avg = no_promo_stores['Customers'].mean()

#create a bar plot to compare the average Sales and Customers for stores that participated in Promo and those that did not
data = {'Sales': [promo_sales_avg, no_promo_sales_avg], 'Customers': [promo_customers_avg, no_promo_customers_avg]}
df_plot = pd.DataFrame(data, index=['Promo', 'No Promo'])

#set the plot size
fig, ax = plt.subplots(figsize=(6, 4))

#create the bar plot
df_plot.plot(kind='bar', ax=ax, rot=0)

#set the plot title and axis labels
ax.set_title('Average Sales and Customers for Promo and No Promo Stores')
ax.set_xlabel('Promo Status')
ax.set_ylabel('Average')

#display the plot
plt.show()

In [ ]:
#compare sales and customers for promo and non-promo stores

#subset the data to include only stores that participated in Promo and those that did not
promo_stores = df[df['Promo2'] == 1]
no_promo_stores = df[df['Promo2'] == 0]

#calculate the average Sales and average Customers for stores that participated in Promo and those that did not
promo_sales_avg = promo_stores['Sales'].mean()
promo_customers_avg = promo_stores['Customers'].mean()
no_promo_sales_avg = no_promo_stores['Sales'].mean()
no_promo_customers_avg = no_promo_stores['Customers'].mean()

#create a bar plot to compare the average Sales and Customers for stores that participated in Promo and those that did not
data = {'Sales': [promo_sales_avg, no_promo_sales_avg], 'Customers': [promo_customers_avg, no_promo_customers_avg]}
df_plot = pd.DataFrame(data, index=['Promo2', 'No Promo2'])

#set the plot size
fig, ax = plt.subplots(figsize=(6, 4))

#create the bar plot
df_plot.plot(kind='bar', ax=ax, rot=0)

#set the plot title and axis labels
ax.set_title('Average Sales and Customers for Promo2 and No Promo2 Stores')
ax.set_xlabel('Promo2 Status')
ax.set_ylabel('Average')

#display the plot
plt.show()

In [ ]:
#compute the average Sales and average Customers for each Store that participated in Promo
promo_store_stats = df[df['Promo'] == 1].groupby('Store')[['Sales', 'Customers']].mean().reset_index()

#compute the average Sales and average Customers for each Store that did not participate in Promo
no_promo_store_stats = df[df['Promo'] == 0].groupby('Store')[['Sales', 'Customers']].mean().reset_index()

#create a scatter plot of average Sales vs. average Customers for the two groups, with each bubble indicating a Store
fig, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(data=promo_store_stats, x='Sales', y='Customers', label='Promo', ax=ax)
sns.scatterplot(data=no_promo_store_stats, x='Sales', y='Customers', label='No Promo', ax=ax)

#set the plot title and axis labels
plt.title('Average Sales vs. Average Customers by Promo Participation')
plt.xlabel('Average Sales')
plt.ylabel('Average Customers')

plt.show()

In [ ]:
#compute the average Sales and average Customers for each Store that participated in Promo
promo_store_stats = df[df['Promo2'] == 1].groupby('Store')[['Sales', 'Customers']].mean().reset_index()

#compute the average Sales and average Customers for each Store that did not participate in Promo
no_promo_store_stats = df[df['Promo2'] == 0].groupby('Store')[['Sales', 'Customers']].mean().reset_index()

#create a scatter plot of average Sales vs. average Customers for the two groups, with each bubble indicating a Store
fig, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(data=promo_store_stats, x='Sales', y='Customers', label='Promo2', ax=ax)
sns.scatterplot(data=no_promo_store_stats, x='Sales', y='Customers', label='No Promo2', ax=ax)

#set the plot title and axis labels
plt.title('Average Sales vs. Average Customers by Promo2 Participation')
plt.xlabel('Average Sales')
plt.ylabel('Average Customers')

plt.show()

In [ ]:
# Count the number of unique stores for Promo2 = 1
promo2_1_stores = df[df['Promo2'] == 1]['Store'].nunique()

# Count the number of unique stores for Promo2 = 0
promo2_0_stores = df[df['Promo2'] == 0]['Store'].nunique()

print("Number of unique stores with Promo2 = 1: ", promo2_1_stores)
print("Number of unique stores with Promo2 = 0: ", promo2_0_stores)

In [ ]:
#view store 1 and sort by date
df_store_1 = df.loc[df['Store']==1]
df_store_1.sort_values('Date', inplace=True)

In [ ]:
#create the line plot
sns.set_style("darkgrid")
plt.figure(figsize=(30, 6))
sns.lineplot(data=df_store_1, x="Date", y="Sales")

#add vertical lines where Promo occurred
promo_dates = df_store_1.loc[df_store_1["Promo"] == 1, "Date"]
for date in promo_dates:
    plt.axvline(x=date, color="red", linestyle="--")

#set plot labels and title
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("Sales Over Time (with Promo Dates)")

#display the plot
plt.show()

In [ ]:
#filter the dataset to only include dates in 2013
df_store_1_2013 = df_store_1[df_store_1['Date'].dt.year == 2013]

#create the line plot
sns.set_style("darkgrid")
plt.figure(figsize=(30, 6))
sns.lineplot(data=df_store_1_2013, x="Date", y="Sales")

#add vertical lines where Promo occurred
promo_dates = df_store_1_2013.loc[df_store_1_2013["Promo"] == 1, "Date"]
for date in promo_dates:
    plt.axvline(x=date, color="red", linestyle="--")

#set plot labels and title
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("Sales over Time (with Promo Dates) for 2013")

#display the plot
plt.show()

In [ ]:
#view store 2 and sort by date
df_store_2 = df.loc[df['Store']==2]
df_store_2.sort_values('Date', inplace=True)

#filter the dataset to only include dates in 2013
df_store_2_2013 = df_store_2[df_store_2['Date'].dt.year == 2013]

#create the line plot
sns.set_style("darkgrid")
plt.figure(figsize=(30, 6))
sns.lineplot(data=df_store_2_2013, x="Date", y="Sales")

#add vertical lines where Promo occurred
promo_dates = df_store_2_2013.loc[df_store_2_2013["Promo"] == 1, "Date"]
for date in promo_dates:
    plt.axvline(x=date, color="red", linestyle="--")

#set plot labels and title
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("Sales over Time (with Promo Dates) for 2013")

#display the plot
plt.show()

In [ ]:
#view store 2 and sort by date
df_store_2 = df.loc[df['Store']==2]
df_store_2.sort_values('Date', inplace=True)

#filter the dataset to only include dates from 1/1/2013 to 6/30/2013
start_date = pd.Timestamp('2013-01-01')
end_date = pd.Timestamp('2013-06-30')
df_store_2_2013 = df_store_2[(df_store_2['Date'] >= start_date) & (df_store_2['Date'] <= end_date)]

#create the line plot
sns.set_style("darkgrid")
plt.figure(figsize=(30, 6))
sns.lineplot(data=df_store_2_2013, x="Date", y="Sales")

#add vertical lines where Promo occurred
promo_dates = df_store_2_2013.loc[df_store_2_2013["Promo"] == 1, "Date"]
for date in promo_dates:
    plt.axvline(x=date, color="red", linestyle="--")

#set plot labels and title
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("Sales over Time (with Promo Dates) for 2013 (6-months)")

#display the plot
plt.show()

In [ ]:
#view store 2 and sort by date
df_store_2 = df.loc[df['Store']==2]
df_store_2.sort_values('Date', inplace=True)

#filter the dataset to only include dates from 1/1/2013 to 3/31/2013
start_date = pd.Timestamp('2013-01-01')
end_date = pd.Timestamp('2013-03-31')
df_store_2_2013 = df_store_2[(df_store_2['Date'] >= start_date) & (df_store_2['Date'] <= end_date)]

#create the line plot
sns.set_style("darkgrid")
plt.figure(figsize=(30, 6))
sns.lineplot(data=df_store_2_2013, x="Date", y="Sales")

#add vertical lines where Promo occurred
promo_dates = df_store_2_2013.loc[df_store_2_2013["Promo"] == 1, "Date"]
for date in promo_dates:
    plt.axvline(x=date, color="red", linestyle="--")

#set plot labels and title
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("Sales over Time (with Promo Dates) for 2013 (3-months)")

#display the plot
plt.show()

### Review Seasonality

In [ ]:
#create line chart by season
sns.set_style('darkgrid')
plt.figure(figsize=(30, 6))
sns.lineplot(data=df, x='Date', y='Sales', hue='Season', palette='bright')

#set plot title and axis
plt.title('Sales by Season')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
#compute the average sales for each season
seasonal_sales = df.groupby('Season')['Sales'].mean().reset_index()

#create a bar plot of average sales by season
sns.set_style('whitegrid')
sns.barplot(data=seasonal_sales, x='Season', y='Sales', color='blue')

#set the plot title and axis labels
plt.title('Average Sales by Season')
plt.xlabel('Season')
plt.ylabel('Average Sales')

plt.show()

### Review Competition

In [ ]:
# filter data for years 1990 to 2020
filtered_data = df_store[(df_store['CompetitionOpenSinceYear'] >= 1990) & (df_store['CompetitionOpenSinceYear'] <= 2020)]

# filter data for Competition Distance <= 40000
filtered_data2 = df_store[df_store['CompetitionDistance'] <= 40000]

# create a figure with 3 subplots in a row
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# first subplot: distribution of Competition Open Since Year (filtered)
sns.distplot(filtered_data['CompetitionOpenSinceYear'], ax=axs[0])
axs[0].set_title('Distribution of Competition Open Since Year')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Density')

# second subplot: distribution of Competition Open Since Month
sns.distplot(df_store['CompetitionOpenSinceMonth'], ax=axs[1])
axs[1].set_title('Distribution of Competition Open Since Month')
axs[1].set_xlabel('Month')
axs[1].set_ylabel('Density')

# third subplot: distribution of Competition Distance (filtered)
sns.distplot(filtered_data2['CompetitionDistance'], ax=axs[2])
axs[2].set_title('Distribution of Competition Distance')
axs[2].set_xlabel('Competition Distance in meters')
axs[2].set_ylabel('Density')
axs[2].set_xlim(right=40000) # set x-axis limit to 40000

plt.tight_layout()
plt.show()

### Correlations

In [ ]:
# create correlation matrix from train data excluding unwanted features
exclude_features = ['DayOfWeek', 'Year', 'Month', 'Day', 'WeekOfYear']
corr_mat = df.drop(exclude_features, axis=1).corr()

# plot heatmap
plt.figure(figsize=(8, 10))
heatmap = sns.heatmap(corr_mat.corr()[['Sales']].sort_values(by='Sales', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Sales', fontdict={'fontsize':18}, pad=16);

In [ ]:
df.head()

## Modeling

### Establish Evaluation Metrics

In [21]:
#RMSLE Function
def RMSLE(y_true, y_pred):
    y_pred_use = y_pred.clip(0)
    rmsle = mean_squared_log_error(y_true, y_pred_use)**(1/2)
    return rmsle

In [22]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))) * 100

In [23]:
def accuracy(y_true, y_pred):
    """Function that returns a table showing RMSE, MAE, R^2 and RMSPE."""
    acc_df = pd.DataFrame(data = {"RMSE": [mean_squared_error(y_true, y_pred)**(1/2)],
                                  "MAE": [mean_absolute_error(y_true, y_pred)],
                                  "R^2": [r2_score(y_true, y_pred)],
                                  "RMSLE": [RMSLE(y_true, y_pred)],
                                  "RMSPE": [rmspe(y_true, y_pred)]})
    display(acc_df.style.hide(axis='index'))

In [24]:
#Tree feature importance
def get_tree_importances(model, data):
    return pd.DataFrame({'Importance': model.feature_importances_}, index=data.columns).sort_values(['Importance'], ascending=False)

### Map Categorical Columns with Ordinal Encoder

In [25]:
#define columns to encode
columns_to_encode = ['StoreType', 'Assortment', 'Season', 'PromoInterval']

#convert all values in the selected columns to strings
df[columns_to_encode] = df[columns_to_encode].astype(str)

#create an instance of the OrdinalEncoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=999)

#fit the encoder on the selected columns
encoder.fit(df[columns_to_encode])

#transform the selected columns using encoder
df[columns_to_encode] = encoder.transform(df[columns_to_encode])

### Drop Columns

In [26]:
df = df.drop(columns=["Date"])

In [27]:
df = df.drop('Customers', axis=1)

### Test-Train Split

In [28]:
df_train_small, df_validation = train_test_split(df, test_size=0.25, random_state=201)

In [29]:
summarize_dataframe(df_train_small)

,Data Type,Missing Values,Unique Values,count,mean,std,min,25%,50%,75%,max
Store,int64,0,1115,762906.0,558.789646,321.894791,1.0,281.00,559.0,838.0,1115.0
DayOfWeek,int64,0,7,762906.0,3.998466,1.997604,1.0,2.00,4.0,6.0,7.0
Sales,int64,0,20842,762906.0,5773.523924,3854.715076,0.0,3725.25,5744.0,7856.0,41551.0
Open,int64,0,2,762906.0,0.829889,0.375731,0.0,1.00,1.0,1.0,1.0
Promo,int64,0,2,762906.0,0.381840,0.485838,0.0,0.00,0.0,1.0,1.0
StateHoliday,int64,0,2,762906.0,0.030573,0.172157,0.0,0.00,0.0,0.0,1.0
SchoolHoliday,int64,0,2,762906.0,0.178704,0.383104,0.0,0.00,0.0,0.0,1.0
Year,int64,0,3,762906.0,2013.832324,0.777130,2013.0,2013.00,2014.0,2014.0,2015.0
Month,int64,0,12,762906.0,5.850920,3.324879,1.0,3.00,6.0,8.0,12.0
Day,int64,0,31,762906.0,15.701991,8.786900,1.0,8.00,16.0,23.0,31.0


In [30]:
summarize_dataframe(df_validation)

,Data Type,Missing Values,Unique Values,count,mean,std,min,25%,50%,75%,max
Store,int64,0,1115,254303.0,557.349972,321.948448,1.0,279.0,557.0,837.0,1115.0
DayOfWeek,int64,0,7,254303.0,3.997963,1.996756,1.0,2.0,4.0,6.0,7.0
Sales,int64,0,16900,254303.0,5774.704113,3835.531078,0.0,3733.0,5744.0,7856.0,38037.0
Open,int64,0,2,254303.0,0.830761,0.374964,0.0,1.0,1.0,1.0,1.0
Promo,int64,0,2,254303.0,0.380538,0.485520,0.0,0.0,0.0,1.0,1.0
StateHoliday,int64,0,2,254303.0,0.030381,0.171634,0.0,0.0,0.0,0.0,1.0
SchoolHoliday,int64,0,2,254303.0,0.178476,0.382914,0.0,0.0,0.0,0.0,1.0
Year,int64,0,3,254303.0,2013.832196,0.778196,2013.0,2013.0,2014.0,2014.0,2015.0
Month,int64,0,12,254303.0,5.834288,3.329721,1.0,3.0,6.0,8.0,12.0
Day,int64,0,31,254303.0,15.705186,8.789868,1.0,8.0,16.0,23.0,31.0


In [31]:
#create X and y arrays for training and validation datasets
X_train = df_train_small.drop(columns=["Sales"])
y_train = df_train_small["Sales"]

X_val = df_validation.drop(columns=["Sales"])
y_val = df_validation["Sales"]

### Random Forest Model

#initialize and fit the model on the training data
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

#predict the target values on the validation data
y_val_pred = rf_model.predict(X_val)

In [34]:
#initialize and fit the model on the training data
rf_model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                 max_depth=20, max_features='auto', max_leaf_nodes=None,
                                 max_samples=None, min_impurity_decrease=0.0,
                                 min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
                                 n_estimators=100, n_jobs=-1, oob_score=False,
                                 random_state=21, verbose=0, warm_start=False)
rf_model.fit(X_train, y_train)

#predict the target values on the validation data
y_val_pred = rf_model.predict(X_val)

In [35]:
accuracy(y_val, y_val_pred)

RMSE,MAE,R^2,RMSLE,RMSPE
1024.999506,629.929790,0.928584,0.155858,inf


In [36]:
pd.DataFrame({'Importance': rf_model.feature_importances_}, index=X_train.columns).sort_values(['Importance'], ascending=False)

,Importance
Open,0.489435
CompetitionDistance,0.102646
Store,0.090986
Promo,0.077423
CompetitionOpenSinceYear,0.039730
DayOfWeek,0.037324
CompetitionOpenSinceMonth,0.032414
StoreType,0.019408
Promo2SinceYear,0.017431
Day,0.017136


### Decision Tree Model

#initialize and fit the model on the training data
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

#predict the target values on the validation data
y_val_pred = dt_model.predict(X_val)

In [38]:
# initialize and fit the model on the training data
dt_model = DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=16,
                                 max_features=None, max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2,
                                 min_weight_fraction_leaf=0.0, random_state=21, splitter='random')
dt_model.fit(X_train, y_train)

# predict the target values on the validation data
y_val_pred = dt_model.predict(X_val)

In [39]:
accuracy(y_val, y_val_pred)

RMSE,MAE,R^2,RMSLE,RMSPE
1644.894122,1013.337299,0.816081,0.232408,inf


In [40]:
pd.DataFrame({'Importance': dt_model.feature_importances_}, index=X_train.columns).sort_values(['Importance'], ascending=False)

,Importance
Open,0.549097
Promo,0.086883
Store,0.066344
CompetitionDistance,0.060514
CompetitionOpenSinceMonth,0.045516
DayOfWeek,0.035516
StoreType,0.032657
CompetitionOpenSinceYear,0.031030
Promo2SinceYear,0.018447
Assortment,0.015627


### XGB Model

In [41]:
# create X and y arrays for training and validation datasets
X_train = df_train_small.drop(columns=["Sales"])
y_train = df_train_small["Sales"]

X_val = df_validation.drop(columns=["Sales"])
y_val = df_validation["Sales"]

# define model parameters
params = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror',
    'random_state': 42
}

# initialize the model
xgb_model = xgb.XGBRegressor(**params)

# train the model on the training data
xgb_model.fit(X_train, y_train)

# predict the target values on the validation data
y_val_pred = xgb_model.predict(X_val)

In [42]:
accuracy(y_val, y_val_pred)

RMSE,MAE,R^2,RMSLE,RMSPE
1704.526381,1180.345005,0.802504,1.517304,inf


In [43]:
pd.DataFrame({'Importance': xgb_model.feature_importances_}, index=X_train.columns).sort_values(['Importance'], ascending=False)

,Importance
Open,0.447405
Promo,0.127730
StateHoliday,0.079144
DayOfWeek,0.064982
Promo2,0.033898
Promo2SinceYear,0.025658
Store,0.024803
StoreType,0.023501
Assortment,0.022142
CompetitionDistance,0.021107


### Light GB Model

#define model parameters
params = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'regression',
    'random_state': 42
}

#initialize the model
lgb_model = lgb.LGBMRegressor(**params)

#train the model on the training data
lgb_model.fit(X_train, y_train)

#predict the target values on the validation data
y_val_pred = lgb_model.predict(X_val)

In [44]:
# initialize the model
lgb_model = lgb.LGBMRegressor(n_leaves=50, 
                  n_estimators=700, 
                  max_depth=-1,  
                  learning_rate=0.3, #0.1
                  subsample=1,
                  colsample_bytree=0.8,
                  reg_alpha=0.1,
                  reg_lambda=1)

# train the model on the training data
lgb_model.fit(X_train, y_train)

# predict the target values on the validation data
y_val_pred = lgb_model.predict(X_val)

[LightGBM] [Warning] Unknown parameter: n_leaves


In [45]:
accuracy(y_val, y_val_pred)

RMSE,MAE,R^2,RMSLE,RMSPE
743.560241,503.683329,0.962418,1.450273,inf


In [46]:
pd.DataFrame({'Importance': lgb_model.feature_importances_}, index=X_train.columns).sort_values(['Importance'], ascending=False)

,Importance
Store,3440
CompetitionDistance,3240
DayOfWeek,2188
CompetitionOpenSinceMonth,1245
Day,1187
WeekOfYear,1185
CompetitionOpenSinceYear,1177
Open,1106
Promo,897
StoreType,843


### Linear Regression Model

In [47]:
# initialize and fit the linear regression model on the training data
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# predict the target values on the validation data
y_val_pred = lr_model.predict(X_val)

In [48]:
accuracy(y_val, y_val_pred)

RMSE,MAE,R^2,RMSLE,RMSPE
2543.007552,1771.111685,0.560412,1.756855,inf


In [49]:
# create a dataframe to hold the feature importance values
df_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': abs(lr_model.coef_)})

# sort the dataframe by importance values in descending order
df_importance = df_importance.sort_values(by=['Importance'], ascending=False)

# reset the index of the dataframe
df_importance = df_importance.reset_index(drop=True)

# print the feature importance values
print(df_importance)

                      Feature    Importance
0                      Promo2  56846.077699
1                        Open   5569.212953
2                       Promo   2080.024105
3                StateHoliday   1091.166099
4                  Assortment    344.645348
5                      Season    174.225658
6                        Year    163.870786
7                   DayOfWeek    140.172291
8               PromoInterval    105.817995
9                       Month     62.485700
10                  StoreType     51.430653
11  CompetitionOpenSinceMonth     38.323945
12              SchoolHoliday     30.155207
13            Promo2SinceYear     28.686405
14            Promo2SinceWeek     16.327543
15                 WeekOfYear      6.501328
16                        Day      2.798275
17   CompetitionOpenSinceYear      0.173394
18        CompetitionDistance      0.024131
19                      Store      0.023870


### Ensemble Model

In [50]:
# Create a copy of the dataset df
df_new = df.copy()

# Drop the Sales column from the new dataset
df_new.drop('Sales', axis=1, inplace=True)

In [51]:
# Predict using the trained models
rf_preds = rf_model.predict(df_new)
dt_preds = dt_model.predict(df_new)
xgb_preds = xgb_model.predict(df_new)
lgb_preds = lgb_model.predict(df_new)
lr_preds = lr_model.predict(df_new)

In [52]:
# Add predictions as new columns in df_train
df['rf_preds'] = rf_preds
df['dt_preds'] = dt_preds
df['xgb_preds'] = xgb_preds
df['lgb_preds'] = lgb_preds
df['lr_preds'] = lr_preds

In [53]:
# Create a new DataFrame for the predictors
preds_df = pd.DataFrame({'rf_preds': rf_preds,
                         'dt_preds': dt_preds,
                         'xgb_preds': xgb_preds,
                         'lgb_preds': lgb_preds,
                         'lr_preds': lr_preds})

# Fit linear regression on the predictors and Sales
lr = LinearRegression()
lr.fit(preds_df, df['Sales'])

# Get the coefficients and intercept
print("Coefficients:", lr.coef_)
print("Intercept:", lr.intercept_)

Coefficients: [ 0.34142224 -0.01198954 -0.14861748  0.80145273  0.01715885]
Intercept: 4.298121967834959


In [72]:
#ensemble model
df['ensemble_ols'] = (4.298121967834959 + 
                      df['rf_preds'] * 0.34142224 + 
                      df['dt_preds'] * -0.01198954 + 
                      df['xgb_preds'] * -0.14861748 + 
                      df['lgb_preds'] * 0.80145273 + 
                      df['lr_preds'] * 0.01715885 )

In [73]:
print("Accuracy of ensemble_ols is:")
accuracy(df["Sales"], df["ensemble_ols"])

Accuracy of ensemble_ols is:


RMSE,MAE,R^2,RMSLE,RMSPE
679.642543,458.694887,0.968836,1.382489,inf


In [74]:
df.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Season,rf_preds,dt_preds,xgb_preds,lgb_preds,lr_preds,ensemble_ols
0,1,5,5263,1,1,0,1,2015,7,31,31,2.0,0.0,1270.0,9.0,2008.0,0,0.0,0.0,0.0,2.0,5410.230870,6540.683616,7335.976562,5882.206774,8288.980411,5539.337253
1,2,5,6064,1,1,0,1,2015,7,31,31,0.0,0.0,570.0,11.0,2007.0,1,13.0,2010.0,2.0,2.0,7052.781958,8178.106383,7432.054199,6348.375112,7518.962307,6426.629232
2,3,5,8314,1,1,0,1,2015,7,31,31,0.0,0.0,14130.0,12.0,2006.0,1,14.0,2011.0,2.0,2.0,8653.598875,8178.106383,8288.290039,8506.783593,7140.907743,8569.307410
3,4,5,13995,1,1,0,1,2015,7,31,31,2.0,2.0,620.0,9.0,2009.0,0,0.0,0.0,0.0,2.0,13217.174039,9206.682243,9938.256836,12711.903076,8994.201542,13271.872326
4,5,5,4822,1,1,0,1,2015,7,31,31,0.0,0.0,29910.0,4.0,2015.0,0,0.0,0.0,0.0,2.0,6027.926500,5194.546099,6479.441406,5956.607908,7893.646435,5946.513384


## Export for Scoring

### Prepare Test Data

In [57]:
#repeat EDA steps for test data

#convert date field
df_test['Date'] = pd.to_datetime(df_test['Date'], format='%Y-%m-%d')

#new fields for year, month, day weekday
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day
df_test['WeekOfYear'] = df_test['Date'].dt.weekofyear

In [58]:
#state holiday conversion
df_test["StateHoliday"] = df_test["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

#merge store and train data
dft = pd.merge(df_test, df_store, how='left', on='Store')

In [59]:
#define season
dft['Season'] = dft['Date'].map(lambda x: 'Winter' if x.month in [12, 1, 2] 
                               else 'Spring' if x.month in [3, 4, 5] 
                               else 'Summer' if x.month in [6, 7, 8] 
                               else 'Fall')

In [60]:
#drop datetime field
dft = dft.drop(columns=["Date"])

In [61]:
#replace nan values from df_store with 0 to indicate no distance or no promo as do not know
dft["CompetitionDistance"].fillna(0, inplace=True)
dft["CompetitionOpenSinceMonth"].fillna(0, inplace=True)
dft["CompetitionOpenSinceYear"].fillna(0, inplace=True)
dft["Promo2SinceWeek"].fillna(0, inplace=True)
dft["Promo2SinceYear"].fillna(0, inplace=True)
dft["PromoInterval"].fillna(0, inplace=True)
dft["Promo"].fillna(0, inplace=True)
dft["Open"] = dft["Open"].fillna(0)
dft["StateHoliday"] = dft["StateHoliday"].fillna(0)

In [62]:
summarize_dataframe(dft)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,int64,0,41088,41088.0,,,,20544.5,11861.228267,1.0,10272.75,20544.5,30816.25,41088.0
Store,int64,0,856,41088.0,,,,555.899533,320.274496,1.0,279.75,553.5,832.25,1115.0
DayOfWeek,int64,0,7,41088.0,,,,3.979167,2.015481,1.0,2.0,4.0,6.0,7.0
Open,float64,0,2,41088.0,,,,0.854094,0.353017,0.0,1.0,1.0,1.0,1.0
Promo,int64,0,2,41088.0,,,,0.395833,0.489035,0.0,0.0,0.0,1.0,1.0
StateHoliday,int64,0,2,41088.0,,,,0.004381,0.066044,0.0,0.0,0.0,0.0,1.0
SchoolHoliday,int64,0,2,41088.0,,,,0.443487,0.496802,0.0,0.0,0.0,1.0,1.0
Year,int64,0,1,41088.0,,,,2015.0,0.0,2015.0,2015.0,2015.0,2015.0,2015.0
Month,int64,0,2,41088.0,,,,8.354167,0.478266,8.0,8.0,8.0,9.0,9.0
Day,int64,0,31,41088.0,,,,13.520833,8.44845,1.0,6.75,12.5,19.25,31.0


In [63]:
#transform the selected columns using encoder
dft[columns_to_encode] = encoder.transform(dft[columns_to_encode])

### Predict Sales with Models

In [64]:
# Select all columns except 'Id'
columns_to_keep = [col for col in dft.columns if col != 'Id']

# Load the new data points into a matrix X of shape (n_samples, n_features)
X = dft[columns_to_keep].values

In [65]:
# Make predictions for the new data points using the trained model
y_pred = rf_model.predict(X)

# Convert the y_pred array to a pandas Series
y_pred_series = pd.Series(y_pred)

# Add the predicted values as a new column 'rf_preds' in the DataFrame
dft['rf_preds'] = y_pred_series

In [66]:
# Make predictions for the new data points using the trained model
y_pred = dt_model.predict(X)

# Convert the y_pred array to a pandas Series
y_pred_series = pd.Series(y_pred)

# Add the predicted values as a new column in the DataFrame
dft['dt_preds'] = y_pred

In [67]:
# Make predictions for the new data points using the trained model
y_pred = xgb_model.predict(X)

# Add the predicted values as a new column in the DataFrame
dft['xgb_preds'] = y_pred

In [68]:
# Make predictions for the new data points using the trained model
y_pred = lgb_model.predict(X)

# Add the predicted values as a new column in the DataFrame
dft['lgb_preds'] = y_pred

In [69]:
# Make predictions for the new data points using the trained model
y_pred = lr_model.predict(X)

# Add the predicted values as a new column in the DataFrame
dft['lr_preds'] = y_pred

In [75]:
#ensemble model
dft['ensemble_ols'] = (4.298121967834959 + 
                      dft['rf_preds'] * 0.34142224 + 
                      dft['dt_preds'] * -0.01198954 + 
                      dft['xgb_preds'] * -0.14861748 + 
                      dft['lgb_preds'] * 0.80145273 + 
                      dft['lr_preds'] * 0.01715885 )

In [76]:
dft.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Season,rf_preds,dt_preds,xgb_preds,lgb_preds,lr_preds,ensemble_ols
0,1,1,4,1.0,1,0,0,2015,9,17,38,2.0,0.0,1270.0,9.0,2008.0,0,0.0,0.0,999.0,0.0,4576.378484,6540.683616,5665.750000,5205.268721,-97451.973984,3145.939271
1,2,3,4,1.0,1,0,0,2015,9,17,38,0.0,0.0,14130.0,12.0,2006.0,1,14.0,2011.0,2.0,0.0,7616.524966,7295.491124,7030.475586,7544.108455,7112.130054,7640.710281
2,3,7,4,1.0,1,0,0,2015,9,17,38,0.0,2.0,24000.0,4.0,2013.0,0,0.0,0.0,999.0,0.0,10821.137253,7469.744408,8765.513672,9337.549629,-97015.699543,8125.534553
3,4,8,4,1.0,1,0,0,2015,9,17,38,0.0,0.0,7520.0,10.0,2014.0,0,0.0,0.0,999.0,0.0,7572.623951,6450.681002,6898.369141,6377.288444,-97537.050625,4924.673007
4,5,9,4,1.0,1,0,0,2015,9,17,38,0.0,2.0,2030.0,8.0,2000.0,0,0.0,0.0,999.0,0.0,9533.337809,7469.744408,6888.207031,7795.821246,-96641.034122,6735.658137


### Output Submission File

#### Output Ensemble Prediction

In [79]:
# select columns to export
cols_to_export = ['Id', 'ensemble_ols']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'ensemble_ols' column to 'Sales'
export_df = export_df.rename(columns={'ensemble_ols': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-ensemble.csv', index=False)

#### Output Random Forest Prediction

In [80]:
# select columns to export
cols_to_export = ['Id', 'rf_preds']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'ensemble_ols' column to 'Sales'
export_df = export_df.rename(columns={'rf_preds': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-rf.csv', index=False)

#### Output Decision Tree Prediction

In [81]:
# select columns to export
cols_to_export = ['Id', 'dt_preds']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'dt_preds' column to 'Sales'
export_df = export_df.rename(columns={'dt_preds': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-dt.csv', index=False)

#### Output XGB Prediction

In [82]:
# select columns to export
cols_to_export = ['Id', 'xgb_preds']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'xgb_preds' column to 'Sales'
export_df = export_df.rename(columns={'xgb_preds': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-xgb.csv', index=False)

#### Output LGB Prediction

In [83]:
# select columns to export
cols_to_export = ['Id', 'lgb_preds']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'lgb_preds' column to 'Sales'
export_df = export_df.rename(columns={'lgb_preds': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-lgb.csv', index=False)

#### Output Linear Regression Prediction

In [84]:
# select columns to export
cols_to_export = ['Id', 'lr_preds']

# create new dataframe with selected columns
export_df = dft[cols_to_export]

# rename the 'lr_preds' column to 'Sales'
export_df = export_df.rename(columns={'lr_preds': 'Sales'})

# export dataframe to CSV file
export_df.to_csv('sales_data-lr.csv', index=False)